In [14]:
from PIL import Image

import pytesseract
import glob, os
from pprint import pprint
import requests

from dotenv import load_dotenv
load_dotenv()



True

In [10]:
screenshots = glob.glob("data/all_commands/*.png")

commands = []

for shot in sorted(screenshots):
  print("Processing: " + shot)
  out = pytesseract.image_to_string(Image.open(shot))
  commands += out.split("\n")

commands = [command for command in commands if len(command) > 3]

# Remove duplicates from commands
commands = list(set(commands))

pprint(commands)
len(commands)

Processing: data/all_commands/Screen Shot 2022-03-30 at 11.42.04 AM.png
Processing: data/all_commands/Screen Shot 2022-03-30 at 11.42.12 AM.png
Processing: data/all_commands/Screen Shot 2022-03-30 at 11.42.22 AM.png
Processing: data/all_commands/Screen Shot 2022-03-30 at 11.42.35 AM.png
Processing: data/all_commands/Screen Shot 2022-03-30 at 11.43.06 AM.png
Processing: data/all_commands/Screen Shot 2022-03-30 at 11.43.14 AM.png
Processing: data/all_commands/Screen Shot 2022-03-30 at 11.43.22 AM.png
Processing: data/all_commands/Screen Shot 2022-03-30 at 11.43.33 AM.png
Processing: data/all_commands/Screen Shot 2022-03-30 at 11.43.41 AM.png
Processing: data/all_commands/Screen Shot 2022-03-30 at 11.43.48 AM.png
Processing: data/all_commands/Screen Shot 2022-03-30 at 11.44.00 AM.png
Processing: data/all_commands/Screen Shot 2022-03-30 at 11.44.09 AM.png
Processing: data/all_commands/Screen Shot 2022-03-30 at 11.44.17 AM.png
Processing: data/all_commands/Screen Shot 2022-03-30 at 11.44.25

426

In [49]:
token = os.environ['HUGGING_FACE_TOKEN']

API_URL = "https://api-inference.huggingface.co/models/sentence-transformers/all-MiniLM-L6-v2"
headers = {"Authorization": f"Bearer {token}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

def top_commands(instruction):
	# This is much slower locally than it would be in production because we're
	# re-computing the embeddings for each command every time, instead of caching
	# them and only computing the embedding for the user's instruction.

	output = query({
		"inputs": {
			"source_sentence": instruction,
			"sentences": commands
		},
	})
	if ('error' in output):
		raise Exception(output['error'])

	with_probabilities = list(zip(commands, output))

	print(f"\nTop matches for instruction `{instruction}`:")
	for command, probability in sorted(with_probabilities, key=lambda x: x[1], reverse=True)[:10]:
		print(f"\t{command.ljust(30)}: {probability:.2f}")

	return sorted(with_probabilities, key=lambda x: x[1], reverse=True)

top_commands("new file")
None



Top matches for instruction `new file`:
	Create: New File...           : 0.82
	File: New Untitled File       : 0.79
	File: Open File...            : 0.63
	File: Save                    : 0.60
	File: Open...                 : 0.60
	File: Save As...              : 0.57
	File: Open Active File in New Window: 0.55
	File: Open Folder...          : 0.55
	File: Open Recent...          : 0.53
	Go to File...                 : 0.50


In [51]:

top_commands("cleanup imports")
top_commands("open in another pane")
top_commands("split the pane")
top_commands("add package")
None


Top matches for instruction `cleanup imports`:
	Organize Imports              : 0.73
	Delete All Left               : 0.31
	Delete All Right              : 0.31
	Java: Delete                  : 0.31
	Unfold All                    : 0.28
	Keep Editor                   : 0.27
	: Close All Editors           : 0.27
	Unpin Editor                  : 0.27
	Compile All                   : 0.25
	Close Editor                  : 0.25

Top matches for instruction `open in another pane`:
	Open Next Editor              : 0.51
	Open Next Editor in Group     : 0.51
	Open Definition to the Side   : 0.48
	Open Previous Editor          : 0.48
	Open Previous Editor in Group : 0.47
	File: Open Active File in New Window: 0.47
	Open Last Editor in Group     : 0.46
	Quick Open Previous Recently Used Editor in Group: 0.43
	Select Next in Quick Open     : 0.43
	Close Window                  : 0.42

Top matches for instruction `split the pane`:
	Split Editor                  : 0.53
	Notebook: Split Cell        